In [ ]:
! pip install dipy SimpleITK patchify


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd '/content/drive'

Mounted at /content/drive
/content/drive


In [ ]:
!pwd

/content/drive


In [ ]:
%cd '/content/drive/MyDrive/newibsr/IBSR/IBSR_nifti_stripped'


/content/drive/MyDrive/newibsr/IBSR/IBSR_nifti_stripped


In [ ]:
from augmented_both_stacked_datagen_wo_patchify import load_nifti, dataGen
from k_imageGen_stacked import ImageHelper, read_image
from alwaysnextdensenet_demo import TrainHistory
from functools import partial
from keras import backend as K
import tensorflow
def label_wise_dice_coefficient(y_true, y_pred, label_index):
    return dice_coef(y_true[:,:,:,:,label_index], y_pred[:,:, :,:,label_index])
    
def get_label_dice_coefficient_function(label_index):
    f = partial(label_wise_dice_coefficient, label_index=label_index)
    f.__setattr__('__name__', 'label_{0}_dice_coef'.format(label_index))
    return f  
def dice_coef(y_true, y_pred, epsilon=1e-6):
    
    axes = tuple(range(1, len(y_pred.shape)-1)) 
    numerator = 2. * K.sum(K.abs(y_pred * y_true), axes)
    denominator = K.sum(K.square(y_pred) + K.square(y_true), axes)
    return K.mean((numerator + epsilon) / (denominator + epsilon))
    
def dice_coef_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)

def weighted_categorical_crossentropy(weights):
    weights = tensorflow.keras.backend.variable(weights)
    def loss(y_true, y_pred):

        y_pred /= tensorflow.keras.backend.sum(y_pred, axis=-1, keepdims=True)
        # clip to prevent NaN's and Inf's
        y_pred = tensorflow.keras.backend.clip(y_pred, tensorflow.keras.backend.epsilon(), 1 - tensorflow.keras.backend.epsilon())
        # calc
        loss = y_true * tensorflow.keras.backend.log(y_pred) * weights
        loss = -tensorflow.keras.backend.sum(loss, -1)
        return loss
        #print("Done.")
    return loss
! pip install -U keras-tuner

Requirement already up-to-date: keras-tuner in /usr/local/lib/python3.6/dist-packages (1.0.2)


In [ ]:
!pwd
from both_stacked_densenet3d import get_model

/content/drive/My Drive/newibsr/IBSR/IBSR_nifti_stripped


In [ ]:
#import tifffile as tiff
import matplotlib.pyplot as plt
from glob import glob
import os
import numpy as np
from collections import OrderedDict
import time
#import h5py

import tensorflow as tf
import tensorflow.keras
from tensorflow.keras import Model,regularizers
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Dropout, Conv2DTranspose, concatenate, SeparableConv2D, AveragePooling2D, UpSampling2D
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping
from alwaysnextdensenet_demo import TrainHistory

from kerastuner.tuners import RandomSearch
from kerastuner.engine.hypermodel import HyperModel
from kerastuner.engine.hyperparameters import HyperParameters
print(tf.version.VERSION)
#from datagen import dataGen
# from k_imageGen8_3 import getImage, one_hot_labels, read_image
#from k_imageGen_stacked import ImageHelper, read_image
#from tensorflow.keras.backend import manual_variable_initialization
#from newoptimizer import AdaBoundOptimizer
#from densenet3d import get_model
#import gc
# def getregstrength(model):
#     for idx,layer in enumerate(model.layers):
#         if(layer.__class__.__name__ == 'Conv2D'):
#             return model.get_config()['layers'][idx]['config']['kernel_regularizer']['config']['C']
#             break


#if os.path.exists('D:/IBSR_nifti_stripped/block_list_dense.csv') and False :
#    os.remove('D:/IBSR_nifti_stripped/block_list_dense.csv')


def getHyperParameters(regstrength,block):
    hp = HyperParameters()
    # hp.Choice('layers', [5,6,7])
    hp.Fixed('layers',block)
    # hp.Choice('is_simple', [0,1])
    hp.Fixed('is_simple', 1)
    # hp.Choice('features_root', [32, 64])  
    hp.Fixed('features_root',32)
    # hp.Choice('learning_rate', [1e-5,1e-6])
    hp.Fixed('learning_rate', 1e-4)
    # hp.Choice('regstrength',[0.001,0.005,0.010])
    hp.Fixed('regstrength', regstrength)
    # hp.Choice('pool_size', [2,4])
    hp.Fixed('pool_size', 2)
    return hp
    #returns hyper parameter object for current regulerization strength value

def getAuxModelsList(modelno,regidx,regstrengthval,patch_size,suffix):
    
    modelist=[]
    f = open('/content/drive/MyDrive/newibsr/IBSR/IBSR_nifti_stripped/block_list_dense'+suffix+'.csv',"r")
    block_list=np.round(np.loadtxt(f,delimiter=','))
    f.close()
    inputlayer = None
    previousoutputlayer = None
    for itr in range (0,modelno):
        #print("inside",'\n')
        #f = open(f'/tf/workspace/stackedvalues/valandhpblock_{itr}.csv',"r")
        #regstrengthval = np.loadtxt(f,delimiter=',')[1]
        #f.close()
        if(modelno==1 and regidx==0):
            block = block_list
            print("in block is ",block)
        else:
            block = block_list[itr]
            print("in block list is ",block_list)

        
        
        hyperparams = getHyperParameters(regstrengthval,int(block))

        if(itr > 0):
            mlist = [md.layers[-1].output for md in modelist]
            previousoutputlayer = mlist[0]
            if len(mlist)>1:
                previousoutputlayer = tf.keras.layers.Add()(mlist)
            

        currmodel = get_model(hyperparams,patch_size,itr,True,False,inputlayer,previousoutputlayer)
       
        if(itr == 0):
            inputlayer = currmodel.input
      
        for layer in currmodel.layers:
            layer.trainable = False
            
        
        modelist.append(currmodel)
        

    return modelist,inputlayer


2.4.0


In [ ]:
import datetime as dt
ts=dt.datetime.now()
addit = '_'+str(ts.day)+'_'+str(ts.hour)+'_'+str(ts.minute)+'_'+str(ts.second)
suffix='_s1'+addit
tile_size = 64
kernel_size = 3
in_channels = 1
out_channels = 4
up_sampling_stride = 2
# dropout_rate = 0.3
label_wise_dice_metrics = [get_label_dice_coefficient_function(index) for index in range(out_channels)]
batch_size = 3

# learning_rate = 0.00001
weights = [0.44459948872961297, 0.993399472922611, 0.7146549346480165, 0.8473461036997595]
#    color_to_label={(0,0,255):0, (0,255,0):1, (0,255,255):2, (255,0,0):3, (255,255,0):4, (255,255,255):5}

image_height = 256
image_width = 256
image_depth = 256

path_train_x = 'dataset_trainboth1.json'

path_val_x = 'dataset_testboth1.json'


# model = get_model(tile_size, kernel_size, in_channels,
#         out_channels, layers, features_root,
#         up_sampling_stride,
#         learning_rate)

e_tile_size = tile_size // 2
n_images_col = len([i for i in range(0, image_width, e_tile_size) if (i + tile_size) < image_width]) + 1
n_images_row = len([i for i in range(0, image_height, e_tile_size) if (i + tile_size) < image_height]) + 1
n_images_depth = len([i for i in range(0, image_depth, e_tile_size) if (i + tile_size) < image_depth]) + 1
print("row and column and depth images: ", n_images_row, " ", n_images_col, " ", n_images_depth)                

# train_ids = [filename.split('/')[-1] for filename in glob(os.path.join(path_train_x, '*.tif'))]
# test_ids = [filename.split('/')[-1] for filename in glob(os.path.join(path_val_x, '*.tif'))]
# print(len(train_ids), "  ", len(test_ids))

n_tiles_train = ( n_images_col * n_images_row * n_images_depth ) * 10
n_tiles_test = ( n_images_col * n_images_row * n_images_depth ) * 8
print("n_tiles_train: ", n_tiles_train, "\nn_tiles_test: ", n_tiles_test)

train_spe = (n_tiles_train // batch_size) if (n_tiles_train % batch_size == 0) else ( (n_tiles_train // batch_size) + 1 )
validation_spe = (n_tiles_test // batch_size) if (n_tiles_test % batch_size == 0) else ( (n_tiles_test // batch_size) + 1 )
print("t_spe: ", train_spe, "\nv_spe: ", validation_spe)

#ih = ImageHelper(batch_size = batch_size, tile_size = tile_size, color_to_label = color_to_label)
#ih.fit_stats(path_train_x, sample_images = 5)
train_spe = 234
validation_spe = 207

# train_spe = 15
# validation_spe=5

epch=100

#------------------------------------------------------------------------------------------------------------------------------------------------

number_of_stacked_models = 2
n_cycles=6
reglist = [5e-4]
block = 4
patch_size = 64
step_size = 32
modelitr=0
while True:
  if(modelitr ==  number_of_stacked_models):
      break
  # if modelitr > 0 :
  #     block=block+2

  print("Model itr is ",modelitr,'\n')

  if(modelitr == 0 ):
      for idx,k in enumerate(reglist):
          print("Reglist is ",k,'\n')
          
          hype = getHyperParameters(k,block)
          currmodel = get_model(hype,patch_size,modelitr,isaux = False,dummy = False)
          print(currmodel.summary)
        
          # traingen = ih.getImage(image_path = path_train_x, label_path = path_train_y,num_of_models = modelitr+1)
          # testgen = ih.getImage(image_path = path_val_x, label_path = path_val_y,num_of_models = modelitr+1)
          traingen = dataGen(batch_size,patch_size,step_size,path_train_x,modelitr+1)
          testgen = dataGen(batch_size,patch_size,step_size,path_val_x,modelitr+1)
                  
          currmodel.fit(traingen, steps_per_epoch = train_spe,
                      validation_data = testgen, validation_steps = validation_spe,
                      epochs = epch,
                      callbacks = [EarlyStopping('val_loss', patience = 100),TrainHistory(modelitr,k,block,idx,epch, n_cycles,1e-3,5e-4,suffix,False)]
                      )
          #TrainHistory(modelitr,k,block,idx,epch, n_cycles,1e-3,4e-4)
  else:
      temp = modelitr-1
      
      

      for idx,k in enumerate(reglist):
          print("Reglist is ",k,'\n')
          hype = getHyperParameters(k,block)
          

        

          modelreturn,inputlayer = getAuxModelsList(modelitr,idx,k,patch_size,suffix)
         
          if(len(modelreturn)>1):
              
              previousoutputlayer = tf.keras.layers.Add()([md.layers[-1].output for md in modelreturn])
          else:
              previousoutputlayer = modelreturn[0].output

          currmodel = get_model(hype,patch_size,modelitr,True,False,inputlayer,previousoutputlayer)
          
      
          allmodellist = modelreturn + [currmodel]
          
          curroutput = tf.keras.layers.Add()([md.layers[-1].output for md in allmodellist])

      
          curroutput = tf.keras.layers.Activation('softmax')(curroutput)

          
          

          
          finalmodel = Model(inputs = allmodellist[-1].input, outputs = curroutput)
      

          finalmodel.compile(optimizer =SGD(lr = hype.get('learning_rate'),momentum = 0.99,nesterov = True), loss = weighted_categorical_crossentropy(weights) , metrics = ["accuracy", dice_coef, label_wise_dice_metrics])
          #optimizer = SGD(lr = hype.get('learning_rate'),momentum=0.9,nesterov=True)
          #loss = "categorical_crossentropy"
          finalmodel.load_weights(f'/content/drive/MyDrive/newibsr/IBSR/IBSR_nifti_stripped/stackedweights_modelsave/modelnoblock_{modelitr-1}_dense'+suffix+'.h5',by_name = True)

          traingen = dataGen(batch_size,patch_size,step_size,path_train_x,modelitr+1)
          testgen = dataGen(batch_size,patch_size,step_size,path_val_x,modelitr+1)

          finalmodel.fit(traingen, steps_per_epoch = train_spe,
                      validation_data = testgen, validation_steps = validation_spe,
                      epochs = epch,
                      callbacks = [EarlyStopping('val_loss', patience = 100),TrainHistory(modelitr,k,block,idx,epch, n_cycles,1e-3,5e-4,suffix,False)])
                
  modelitr= modelitr+1


row and column and depth images:  7   7   7
n_tiles_train:  3430 
n_tiles_test:  2744
t_spe:  1144 
v_spe:  915
Model itr is  0 

Reglist is  0.0005 

block1_sel_0
block2_sel_0
block3_sel_0
block4_sel_0
Model: "model"
______________________________________________________________________________________________________________________________________________________
Layer (type)                                     Output Shape                     Param #           Connected to                                      
input_1 (InputLayer)                             [(None, 64, 64, 64, 2)]          0                                                                   
______________________________________________________________________________________________________________________________________________________
block1_sel_0 (Conv3D)                            (None, 64, 64, 64, 32)           96                input_1[0][0]                                     
___________________________